**Sarah Graber**<br>
sg3518<br>
Assignment 2<br>
Collaborators: Emily Zhang

In [123]:
import math
import numpy as np
import random

## Problem 1
using trapezoidal rule

In [124]:
def integraltr(func, lowbound, upbound, N):
    """gives the integral of sin(x) with givens: lower bound, upper bound, number of rectangles"""
   
    # steps through the given bounds in N steps, finds the area of each rectangle + each trapezoid, sums them
    sum=0
    for x in np.linspace(lowbound, upbound, N, endpoint=False):
        width = ((upbound-lowbound)/N)
        height_rectangle = func(x)
        height_trapezoid = func(x+width)
        rectangle_trapezoid = (height_rectangle*width) + (((height_trapezoid - height_rectangle) * width) / 2)
        sum = sum + rectangle_trapezoid
    return sum

integraltr(math.sin,0,math.pi/2,10)

0.9979429863543573

In [125]:
# applying the integral function for each power of 10
intervals = [10,100,1000,10000,100000]
for i in intervals:
    error = abs((integraltr(math.sin,0,math.pi/2,i))-1)/1
    print("N=",i,", integral=",integraltr(math.sin,0,math.pi/2,i), ", fractional error=",error)
    
print("")
print("error decreases 10x as fast as with the rectangle rule!")

N= 10 , integral= 0.9979429863543573 , fractional error= 0.0020570136456427024
N= 100 , integral= 0.9999794382396072 , fractional error= 2.0561760392778794e-05
N= 1000 , integral= 0.9999997943832338 , fractional error= 2.0561676616903668e-07
N= 10000 , integral= 0.9999999979438333 , fractional error= 2.056166681363436e-09
N= 100000 , integral= 0.9999999999794249 , fractional error= 2.057509718156325e-11

error decreases 10x as fast as with the rectangle rule!


## Problem 2

In [53]:
def Flux(p,z,N):
    '''gives the ratio of obscured to unobscured flux of a star during planetary transit w/ limb darkening'''
    
    def I(r):
        '''limb darkening parameter'''
        return 1
        
    # defining the denominator of the formula
    def dformula(r):
        '''function to multiply a variable by 2 * I'''
        return 2*r*I(r)
    
    denom = integraltr(dformula,0,1,N)
    
    # defining the numerator of the formula
    def nformula(r):
        '''function to multiply a variable by 2 * (1-delta) * I'''
        
        # delta partial function; different function for different values of z
        if r>=(z+p) or r<=(z-p):
            delta=0
        elif (r+z)<=p:
            delta=1
        else:
            delta=(math.pi)**(-1)*np.arccos((z**2 - p**2 + r**2)/(2*z*r))
            
        return (1-delta)*2*r*I(r)
    
    num = integraltr(nformula,0,1,N)
    
    flx = num / denom
    
    return flx

In [57]:
# flux formula copied from assignment 1 for comparison; no limb-darkening parameter
def Flux_old(p,z):
    
    # first make sure z is taken as the absolute value
    z=abs(z)
    
    # lambda partial function; different function for different values of z
    if (1+p)<z:
        lamda=0    #misspelled lambda bc the correct spelling is some native python function
    elif abs(1.-p)<z and z<=(1.+p):
        # define kappa 0 and 1 wrt p,z
        kap1 = math.acos( (1 - p**2 + z**2) / (2 * z) )
        kap0 = math.acos((p**2+z**2-1)/(2*p*z))
        lamda=(1/math.pi)*(p**2 * kap0 + kap1 - math.sqrt((4*z**2 - (1+z**2 -p**2)**2)/4))
    elif z<=(1.-p):
        lamda=p**2
    elif z<= (p-1):
        lamda=1
        
    #final observed flux wrt unobscured flux
    return(1 - lamda)

In [73]:
# applying the Flux function integrated for each power of 10
intervals = [10,100,1000,10000,100000]
for i in intervals:
    error = abs((Flux_old(0.2,0.9))-Flux(0.2,0.9,i))/Flux_old(0.2,0.9)
    print("N =",i,", Flux =",Flux(0.2,0.9,i), ", fractional error =",error)

N = 10 , Flux = 0.971007549763638 , fractional error = 0.002675030321695045
N = 100 , Flux = 0.9684935353994175 , fractional error = 7.90263776602542e-05
N = 1000 , Flux = 0.9684193666885152 , fractional error = 2.43880172332759e-06
N = 10000 , Flux = 0.9684170790050751 , fractional error = 7.651003645587889e-08
N = 100000 , Flux = 0.968417007248545 , fractional error = 2.4133097376334382e-09


## Problem 3

In [109]:
def MonteCarlo_Flux(p,z,N):
    '''gives ratio of obscured flux to unobscured flux during planetary transit by the monte-carlo integration method'''
    
    N1=0 # number of 'accepted' points that fall inside the unit circle
    N2=0 # number of points that fall inside the circle of the planet
    
    i=0
    while i < N:
        x = random.uniform(-1,1)
        y = random.uniform(-1,1)
    
        if (x**2+y**2)<1:
            N1=N1+1
        if ((x-z)**2+y**2)<p**2:
            N2=N2+1
        i=i+1
        
    flx = (N1 - N2)/N1
    return flx

In [122]:
# applying the Monte-Carlo Flux function integrated for each power of 10
intervals = [10,100,1000,10000,100000]
for i in intervals:
    error = abs((Flux_old(0.2,0.9))-MonteCarlo_Flux(0.2,0.9,i))/Flux_old(0.2,0.9)
    print("N =",i,", Flux =",MonteCarlo_Flux(0.2,0.9,i), ", fractional error =",error)
    
print("")
print("This is a low-order integration method")

N = 10 , Flux = 1.0 , fractional error = 0.03261301167613528
N = 100 , Flux = 0.9736842105263158 , fractional error = 0.023968523210228295
N = 1000 , Flux = 0.9613466334164589 , fractional error = 0.001082843380680713
N = 10000 , Flux = 0.9663091482649843 , fractional error = 0.0007223874854566719
N = 100000 , Flux = 0.9669149952244508 , fractional error = 0.0002555859886556292

This is a low-order integration method
